In [1]:
# importa os pacotes necessários
import numpy as np
from matplotlib import pyplot as plt
from pandas.plotting import scatter_matrix
import seaborn as sns
from scipy import stats as ss
import pandas as pd
import plotly.express as px
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn import tree
import random
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

In [ ]:
# carrega os dados
df = pd.read_csv('dados/BankChurners.csv',low_memory=False)

# head dos dados
df.head()

In [ ]:
# tamanho do dataset
df.shape

In [ ]:
# distribuição da variável target
df.Attrition_Flag.value_counts().plot.pie(autopct='%1.1f%%',figsize=(10,8))

In [ ]:
# remove as colunas que não vamos utilizar
df = df[df.columns[:-2]]

# cria coluna com flag de churn
df['Churn'] = df['Attrition_Flag'].apply(lambda x: 1 if x == 'Attrited Customer' else 0)

df.head()

### 0. Definição de Funções

In [ ]:
def plot_cm(y_true, y_pred, label, x_size=5, y_size=5, savepath=None):
    """
    Plots a confusion matrix

    Params:
        - y_true: the true value of the target
        - y_pred: the predicted value of the data
        - label: the name of the model
        - - x_size: x size of the plot
        - y_size: y size of the plot
        - savepath: the path to save the image
    """
    cm = confusion_matrix(y_true, y_pred, labels=np.unique(y_true))
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = "%.1f%%\n%d/%d" % (p, c, s)
            elif c == 0:
                annot[i, j] = ""
            else:
                annot[i, j] = "%.1f%%\n%d" % (p, c)
    cm = pd.DataFrame(cm, index=np.unique(y_true), columns=np.unique(y_true))
    cm.index.name = "Actual"
    cm.columns.name = "Predicted"
    fig, ax = plt.subplots(figsize=(x_size, y_size))
    plt.title("Matriz de Confusão - " + label, fontsize=15)
    sns.heatmap(cm, cmap="Blues", annot=annot, fmt="", ax=ax)
    # savesfig
    if savepath is not None:
        plt.savefig(savepath + ".png")
    plt.show()
    return cm


def class_def(multi, y_proba):
    "Helper function of the plot_roc_curve function"
    if multi is False:
        return [0]
    else:
        return range(len(y_proba[0]))


# função para plotar a curva ROC
def plot_roc_curve(
    y_true, y_proba, label, x_size=5, y_size=5, savepath=None, strategy="ovo"
):

    """
    Plots the roc auc curve.

    Params:
        - y_true: the true value of the target
        - y_proba: the predicted probability value of the data
        - label: the name of the model
        - - x_size: x size of the plot
        - y_size: y size of the plot
        - savepath: the path to save the image
    """

    # checks if it is multiclass
    if pd.Series(y_true).nunique() > 2:
        multi = True
    else:
        multi = False

    # creates no skill probas
    ns_probs = [0 for _ in range(len(y_true))]
    ns_true = [random.randint(0, 1) for _ in range(len(y_true))]
    ns_auc = roc_auc_score(ns_true, ns_probs)
    ns_fpr, ns_tpr, _ = roc_curve(ns_true, ns_probs)

    # keeps only the positive value if not multiclass
    if multi is False:
        try:
            y_proba = y_proba[:, 1]
        except Exception:
            y_proba = y_proba

    # calculates model ROC AUC SCORE
    if multi is True:
        model_auc = roc_auc_score(
            y_true, y_proba, multi_class=strategy, average="weighted"
        )
    else:
        model_auc = roc_auc_score(y_true, y_proba)

    fpr = {}
    tpr = {}
    thresh = {}
    n_class = class_def(multi, y_proba)
    if multi is True:
        for i in n_class:
            fpr[i], tpr[i], thresh[i] = roc_curve(
                y_true,
                y_proba[:, i],
                pos_label=i
                )
    else:
        for i in n_class:
            fpr[i], tpr[i], thresh[i] = roc_curve(y_true, y_proba)

    # summarize scores
    print("No Skill: ROC AUC=%.3f" % (ns_auc))
    print(label + ": ROC AUC=%.3f" % (model_auc))

    # plots the ROC Curve
    fig, ax = plt.subplots(figsize=(x_size, y_size))

    # define the colors
    colors_ls = ["b", "g", "r", "c", "m", "y", "k", "w"]
    colors = []
    cont = 0
    while len(colors) < len(n_class):
        if cont == len(colors_ls):
            cont = 0
        colors.append(colors_ls[cont])
        cont += 1

    # creates plots
    for i in n_class:
        plt.plot(
            fpr[i],
            tpr[i],
            linestyle="-",
            color=colors[i],
            label=f"Class {i+1}"
            )
    plt.plot(ns_fpr, ns_tpr, linestyle="--", label="No Skill")

    # axis labels
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    # plot title
    plt.title(f"ROC AUC Curve - {label}", fontsize=15)
    # show legend
    plt.legend(loc="best")
    # savesfig
    if savepath is not None:
        plt.savefig(savepath + ".png")
    # show the plot
    plt.show()


def tree_plot(modelo, X, x_size=5, y_size=5, savepath=None):
    """
    Plots the tree for tree based models.

    Params:
        - model: a sklearn trained model
        - X: the input data
        - x_size: x size of the plot
        - y_size: y size of the plot
        - savepath: the path to save the image
    """
    # tamanho do plot
    plt.figure(figsize=(x_size, y_size))

    # árvore criada
    tree.plot_tree(
        decision_tree=modelo,  # modelo criado
        feature_names=X.columns,  # lista de variáveis
        label="all",  # mostra os dados em todos os nós
        filled=True,  # pinta os nós com maior volumetria
        rounded=True,  # plota cada nó com pontas arredondadas
        fontsize=10,  # define o tamanho da fonte
    )

    # savesfig
    if savepath is not None:
        plt.savefig(savepath + ".png")

def clasifiers_test(
            X, Y, modelos, num_folds, metrica, x_size=5, y_size=5, savepath=None):
        """
        This function tests multiple clasifiers with cross validation
        and prints their scores.

        Params:
            - X: the input data
            - Y: the target data
            - num_folds: the number of folds to use
                in cross validation
            - x_size: x size of the plot
            - y_size: y size of the plot
            - savepath: the path to save the image
        """
        # Avaliando cada modelo em um loop
        resultados = []

        nomes = []

        print(f"MODELO\t|\t  MEDIA  \t|\tDESVIO")
        for nome, modelo in modelos:
            kfold = KFold(n_splits=num_folds)
            cv_results = cross_val_score(
                modelo,
                X,
                Y,
                cv=kfold,
                scoring=metrica
                )
            resultados.append(cv_results)
            nomes.append(nome)
            msg = "%s\t|\t%f\t|\t%f" % (
                nome,
                cv_results.mean(),
                cv_results.std()
                )
            print(msg)

        # Boxplot para comparar os algoritmos
        fig = plt.figure(figsize=(x_size, y_size))
        fig.suptitle("Comparação de Algoritmos de Classificação")
        ax = fig.add_subplot(111)
        plt.boxplot(resultados)
        ax.set_xticklabels(nomes)
        # savesfig
        if savepath is not None:
            plt.savefig(savepath + ".png")
        plt.show()
        return pd.DataFrame(
            resultados,
            columns=[f"fold_{i}" for i in range(num_folds)],
            index=nomes
        )

### 1. Definição de Variáveis

In [ ]:
# tipos dos dados
df.dtypes

In [ ]:
# variáveis contínuas
continuas = [
    'Customer_Age',
    'Dependent_count',
    'Months_on_book',
    'Total_Relationship_Count',
    'Months_Inactive_12_mon',
    'Contacts_Count_12_mon',
    'Credit_Limit',
    'Total_Revolving_Bal',
    'Avg_Open_To_Buy',
    'Total_Amt_Chng_Q4_Q1',
    'Total_Trans_Amt',
    'Total_Trans_Ct',
    'Total_Ct_Chng_Q4_Q1',
    'Avg_Utilization_Ratio'
]

# variáveis categóricas
categoricas = [
    'Gender',
    'Education_Level',
    'Marital_Status',
    'Income_Category',
    'Card_Category'
]

# resposta
resposta = ['Churn']

### 2. Análise Exploratória

#### 2.1 Variáveis Contínuas

In [ ]:
df[continuas].describe()

In [ ]:
# faz um plot de densidade das variávels contínuas
df[continuas].plot(
    kind="density",
    subplots=True,
    sharex=False,
    figsize=(10, 10)
    )

Podemos ver que as variáveis contínuas não seguem uma distribuição normal e por isso devemos tomar alguns cuidados na hora de testar uma Regressão Logística por exemplo por ser uma técnica que tem como premissa que as variáveis explicativas seguem uma distribuição normal, o correto seria padronizar os dados antes do treinamento para que a média seja 0 e o desvio padrão 1.

In [ ]:
df[continuas].plot(
        kind="box",
        subplots=True,
        sharex=False,
        figsize=(30, 10)
    )

Podemos agora verificar que algumas das nossas variáveis têm muitos outliers, por isso, é necessário realizar uma padronização antes de utilizarmos um algoritmo que calcula distâncias como o KNN, por exemplo.

In [ ]:
df[continuas + resposta].head()

In [ ]:
# cria matriz de correlação
correlations = df[continuas + resposta].corr()

# cria figura
fig = plt.figure(figsize=(20, 20))

# cria a máscara
mask = np.zeros_like(correlations, dtype=bool)
mask[np.triu_indices_from(mask)] = True

# cria eixos
with sns.axes_style("white"):
    sns.heatmap(
        correlations,
        mask=mask,
        vmin=-1,
        vmax=1,
        square=True,
        annot=True,
        fmt=".2f",
        linewidths=0.5,
        cmap=sns.color_palette("Spectral"),
    )

Podemos ver pelo gráfico acima que algumas variáveis tem uma correlação muito alta entre elas, como por exemplo as variáveis Credit_Limit e Avg_Open_To_Buy que tem uma correlação perfeita e igual a 1.

#### 2.2 Variáveis Categóricas

In [ ]:
# gráfico de distribuição das variáveis categoricas
for col in categoricas:
    df.groupby([col, 'Churn']).size().groupby(level=0).apply(lambda x: 100 * x / x.sum()).unstack().plot(
        kind='bar',
        stacked=True,
        figsize=(5,5),
        title=col
    )

Podemos perceber que a Categoria do Cartão de Crédito, a Categoria de Renda e o Nível Educacional são as variáveis que mais distorcem a distribuição de churn.

### 3. Preparação dos Dados

#### 3.1 Modelagem sem DataPrep

In [ ]:
# separa variáveis preditoras e variável resposta
X = df[categoricas + continuas]
Y = df["Churn"]

# cria os dummies
X = pd.get_dummies(X, drop_first=True)

# cria uma lista de modelos para testar
modelos = []
modelos.append(("LR", LogisticRegression(max_iter=5000)))
modelos.append(("NB", GaussianNB()))
modelos.append(("KNN", KNeighborsClassifier()))
modelos.append(("CART", DecisionTreeClassifier()))
modelos.append(("RF", RandomForestClassifier()))
modelos.append(("SVM", SVC()))
modelos.append(("GB", GradientBoostingClassifier()))

# testa os modelos
clasifiers_test(X=X, Y=Y, modelos = modelos, num_folds=5, metrica="roc_auc")

Neste primeiro teste podemos verificar que os algoritmos Gradient Boosting, Random Forest, Naive Bayes e Regressão Logísitica tiveram melhor performance e também foram mais estáveis tendo um desvio padrão mais baixo.

#### 3.2 Modelagem com DataPrep

In [ ]:
# cria uam cópia do df para o pré-processamento
df_pre = df.copy()

In [ ]:
# cria a correlação
correlacao = df[continuas + resposta].corr()

# lista de pares de variáveis correlacionadas
pares = []

# loop para encontrar as variáveis correlacionadas
for index,row in correlacao.iterrows():
    for col in correlacao.columns:
        # se o valor absoluto for maior que 0.7 e não for a mesma variável appenda o par de variáveis
        if index != col and index != resposta[0] and col != resposta[0] and abs(correlacao.loc[index,col]) > 0.7:
            pares.append((index,col))

# lista de variáveis a serem excluídas
excluir = []

# loop para encontrar as variáveis a serem excluídas
for col1,col2 in pares:
    # se o valor absoluto da correlação da variável com a resposta for maior que o 
    # valor absoluto da correlação da outra variável com a resposta
    # então appenda a variável com menor correlação
    if abs(correlacao.loc[col1,'Churn']) >= abs(correlacao.loc[col2,'Churn']):
        excluir.append(col2)
    else:
        excluir.append(col1)
print(f'Variáveis para excluir por alta correlação: {excluir}')

In [ ]:
# definição do skew maximo
skew_max = 0.5

# calculo do skew da variáveis contínuas
continuas_finais = [col for col in continuas if col not in excluir]
skew = df_pre[continuas_finais].skew()

# cria lista com variáveis que precisam de transformação
skew = skew[abs(skew) > skew_max]
skew = skew.index.tolist()
print(skew)

In [ ]:
# agora vamos normalizar as variáveis para que tenham média 0 e desvio padrão 1
scaler = StandardScaler()
df_pre[skew] = scaler.fit_transform(df_pre[skew])

In [ ]:
# separa variáveis preditoras e variável resposta
X = df_pre[categoricas + continuas]
Y = df_pre["Churn"]

# cria os dummies
X = pd.get_dummies(X, drop_first=True)

# cria uma lista de modelos para testar
modelos = []
modelos.append(("LR", LogisticRegression(max_iter=5000)))
modelos.append(("NB", GaussianNB()))
modelos.append(("KNN", KNeighborsClassifier()))
modelos.append(("CART", DecisionTreeClassifier()))
modelos.append(("RF", RandomForestClassifier()))
modelos.append(("SVM", SVC()))
modelos.append(("GB", GradientBoostingClassifier()))

# testa os modelos
clasifiers_test(X=X, Y=Y, modelos = modelos, num_folds=5, metrica="roc_auc")

Neste segundo teste podemos verificar que os algoritmos que tiveram boa performance no teste anterior continuaram tendo boa performance. As diferenças de fato estão na Regressão Logística e no Naive Bayes, no caso da regressão mesmo utilizando menos variáveis preditoras vemos que teve uma pequena melhora pois no segundo dataset criado as variáveis seguem as premissas do algoritmo, já no caso do Naive Bayes mesmo seguindo as premissas necessárias acabou perdendo um pouco de performance.

Outro ponto interessante é que os demais modelos mantiveram seus resultados mesmo descartando 4 variáveis, mostrando que remover variáveis não necessariamente diminui a performance do modelo.

### 4. Modelagem

#### 4.1 Baseline

Como modelo base neste case vamos utilizar a Regressão Logística por ter alta performance e ser o modelo linear e o Gradient Boosting por ter atingido a mais alta performance dos modelos testados

In [ ]:
# separa variáveis preditoras e variável resposta
X = df_pre[categoricas + continuas]
Y = df_pre["Churn"]

# cria os dummies
X = pd.get_dummies(X, drop_first=True)

# separa os dados entre treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=7)

In [ ]:
# cria o modelo baseline de regressão logística
modelo_rl = LogisticRegression(max_iter=5000)

# treina o modelo
modelo_rl.fit(X_train, Y_train)

# faz as previsões com dados de teste
Y_pred = modelo_rl.predict(X_test)
Y_pred_proba = modelo_rl.predict_proba(X_test)[:, 1]

# faz as previsões com dados de treino
Y_pred_train = modelo_rl.predict(X_train)
Y_pred_proba_train = modelo_rl.predict_proba(X_train)[:, 1]

In [ ]:
# mostra a importância das variáveis
importancia = pd.DataFrame({"Variavel": X_train.columns, "Importancia": modelo_rl.coef_[0]})
importancia.sort_values("Importancia", ascending=False).plot.bar(x="Variavel", y="Importancia")

In [ ]:
# plota a matriz de confusão de teste
plot_cm(Y_test, Y_pred, 'Regressão Logística - Teste')

In [ ]:
# plota a matriz de confusão de treino 
plot_cm(Y_train, Y_pred_train, 'Regressão Logística - Treino')

Podemos perceber que a matriz de confusão ficou com uma acurácia baixa neste modelo porém isso se deve ao fato de termos uma resposta com dados desbalanceados. Para melhor avaliarmos o modelo precisamos plotar a curva roc.

In [ ]:
# plota a curva ROC de teste
plot_roc_curve(Y_test, Y_pred_proba, 'Regressão Logística - Teste')

In [ ]:
# plota a curva ROC de treino
plot_roc_curve(Y_train, Y_pred_proba_train, 'Regressão Logística - Treino')

ROC é uma curva de probabilidade e AUC representa o grau ou medida de separabilidade. Diz o quanto o modelo é capaz de distinguir entre as classes. Quanto maior a AUC, melhor será o modelo.

Aqui atingimos 0.918 em treino e 0.920 em teste, resultados bastante parecidos, o que nos indica que não houve overfitting e por ser um resultado bom também não houve underfitting.

In [ ]:
# plota um histograma com as probabilidades de churn
df_result = pd.DataFrame({'Churn':Y_test, 'Churn_Prob':Y_pred_proba})
sns.histplot(data=df_result,x = 'Churn_Prob',hue='Churn', bins=20, kde=True)

In [ ]:
# plota um histograma com as probabilidades de churn
df_result = pd.DataFrame({'Churn':Y_train, 'Churn_Prob':Y_pred_proba_train})
sns.histplot(data=df_result,x = 'Churn_Prob',hue='Churn', bins=20, kde=True)

Pelos histogramas acima podemos ver que a classe 1 ainda não está tão separada da classe 0 quanto pretendemos

In [ ]:
# cria o modelo baseline de gradient boosting
modelo_gb = GradientBoostingClassifier()

# treina o modelo
modelo_gb.fit(X_train, Y_train)

# faz as previsões com dados de teste
Y_pred = modelo_gb.predict(X_test)
Y_pred_proba = modelo_gb.predict_proba(X_test)[:, 1]

# faz as previsões com dados de treino
Y_pred_train = modelo_gb.predict(X_train)
Y_pred_proba_train = modelo_gb.predict_proba(X_train)[:, 1]

In [ ]:
# mostra a importância das variáveis
importancia = pd.DataFrame({"Variavel": X_train.columns, "Importancia": modelo_gb.feature_importances_})
importancia.query("Importancia > 0").sort_values("Importancia", ascending=False).plot.bar(x="Variavel", y="Importancia")

In [ ]:
# plota a matriz de confusão de teste
plot_cm(Y_test, Y_pred, 'Gradient Boosting - Teste')

In [ ]:
# plota a matriz de confusão de treino
plot_cm(Y_train, Y_pred_train, 'Gradient Boosting - Treino')

Aqui podemos perceber que o Gradient Boosting conseguiu resolver relativamente bem o desbalanceamento da base acertando 89% dos clientes com churn.

In [ ]:
# plota a curva ROC de teste
plot_roc_curve(Y_test, Y_pred_proba, 'Gradient Boosting - Teste')

In [ ]:
# plota a curva ROC de teste
plot_roc_curve(Y_train, Y_pred_proba_train, 'Gradient Boosting - Treino')

Com o Gradient Boosting podemos visualizar que o algoritmo ficou com uma ROC AUC Score bem próximo do perfeito nos resultados de treino e acabou descolando um pouco dos dados de teste, indicando um leve overfitting no mesmo.

In [ ]:
# plota um histograma com as probabilidades de churn
df_result = pd.DataFrame({'Churn':Y_test, 'Churn_Prob':Y_pred_proba})
sns.histplot(data=df_result,x = 'Churn_Prob',hue='Churn', bins=20, kde=True)

In [ ]:
# plota um histograma com as probabilidades de churn
df_result = pd.DataFrame({'Churn':Y_train, 'Churn_Prob':Y_pred_proba_train})
sns.histplot(data=df_result,x = 'Churn_Prob',hue='Churn', bins=20, kde=True)

Nestes histogramas já podemos perceber que há uma clara diferença entre as classes indicando que o modelo consegue separar ambas muito bem.

#### 4.1.1 Ajustando Hiperparâmetros

Nos modelos baseline não ajustamos nenhum hiperparâmetro e podemos melhorar os nossos resultados encontrando o melhor conjunto para cada modelo.

Neste teste vamos alterar os principais hiperparâmetros de cada algoritmo:

- Regressão Logística:
    - Penalty: podemos a penalização de variáveis para melhor ajustar os coeficientes do modelo
    - C: inverso da força de penalização


- Gradient Boosting:
    - learning_rate: taxa de aprendizado do modelo
    - n_estimators: número de árvores de regressão utilizadas pelo modelo
    - min_samples_leaf: quantidade mínima de dados em uma folha
    - max_depth: profundidade máxima das árvores

##### 4.1.1 Gradient Boosting

In [ ]:
# define os hiperparametros
parametros = {
        'learning_rate': [0.01, 0.05, 0.1],
        'n_estimators': [100, 200, 300],
        'min_samples_leaf': [0.05,0.1,0.2],
        'max_depth': [3, 5, 7, 9],
        }

# parametros = {
#         'learning_rate': [0.05, 0.1],
#         'n_estimators': [100],
#         'min_samples_leaf': [0.1,0.2],
#         'max_depth': [ 7, 9],
#         }


# define o modelo
modelo_gb = GradientBoostingClassifier()

# define a estratégia de busca
grid_gb = GridSearchCV(modelo_gb, parametros, cv=5, scoring='roc_auc', n_jobs=-1)

# treina o modelo
grid_gb.fit(X_train, Y_train)

In [ ]:
# imprime os melhores parâmetros
print(f'Os melhores parâmetros encontrados foram: {grid_gb.best_params_}')

# coloca os resultados em um dataframe
resultados = pd.DataFrame(grid_gb.cv_results_)
resultados = resultados.sort_values('mean_test_score', ascending=False)
resultados.head()

In [ ]:
# treina o modelo com os melhores parâmetros
modelo_gb = GradientBoostingClassifier(
    learning_rate=grid_gb.best_params_['learning_rate'],
    max_depth=grid_gb.best_params_['max_depth'],
    min_samples_leaf=grid_gb.best_params_['min_samples_leaf'],
    n_estimators=grid_gb.best_params_['n_estimators']
)

# treina o modelo
modelo_gb.fit(X_train, Y_train)

# faz as previsões com dados de teste
Y_pred = modelo_gb.predict(X_test)
Y_pred_proba = modelo_gb.predict_proba(X_test)[:, 1]

# faz as previsões com dados de treino
Y_pred_train = modelo_gb.predict(X_train)
Y_pred_proba_train = modelo_gb.predict_proba(X_train)[:, 1]

In [ ]:
# plota a matriz de confusão de teste
plot_cm(Y_test, Y_pred, 'Gradient Boosting - Teste')

In [ ]:
# plota a matriz de confusão de treino
plot_cm(Y_train, Y_pred_train, 'Gradient Boosting - Treino')

In [ ]:
# plota a curva ROC de teste
plot_roc_curve(Y_test, Y_pred_proba, 'Gradient Boosting - Teste')

In [ ]:
# plota a curva ROC de teste
plot_roc_curve(Y_train, Y_pred_proba_train, 'Gradient Boosting - Treino')

In [ ]:
# plota um histograma com as probabilidades de churn
df_result = pd.DataFrame({'Churn':Y_test, 'Churn_Prob':Y_pred_proba})
sns.histplot(data=df_result,x = 'Churn_Prob',hue='Churn', bins=20, kde=True)

In [ ]:
# plota um histograma com as probabilidades de churn
df_result = pd.DataFrame({'Churn':Y_train, 'Churn_Prob':Y_pred_proba_train})
sns.histplot(data=df_result,x = 'Churn_Prob',hue='Churn', bins=20, kde=True)

##### 4.1.2 Regressão Logística

In [ ]:
# define os hiperparametros
parametros = {
        'penalty': ['l1','l2','elasticnet'],
        'C': [1,0.5,0.1],
        'max_iter': [5000, 10000, 20000]
        }


# define o modelo
modelo_rl = LogisticRegression()

# define a estratégia de busca
grid_rl = GridSearchCV(modelo_rl, parametros, cv=5, scoring='roc_auc', n_jobs=-1)

# treina o modelo
grid_rl.fit(X_train, Y_train)

In [ ]:
# imprime os melhores parâmetros
print(f'Os melhores parâmetros encontrados foram: {grid_rl.best_params_}')

# coloca os resultados em um dataframe
resultados = pd.DataFrame(grid_rl.cv_results_)
resultados = resultados.sort_values('mean_test_score', ascending=False)
resultados.head()

In [ ]:
# treina o modelo com os melhores parâmetros
modelo_rl = LogisticRegression(
    C=grid_rl.best_params_['C'],
    max_iter=grid_rl.best_params_['max_iter'],
    penalty=grid_rl.best_params_['penalty']
)

# treina o modelo
modelo_rl.fit(X_train, Y_train)

# faz as previsões com dados de teste
Y_pred = modelo_rl.predict(X_test)
Y_pred_proba = modelo_rl.predict_proba(X_test)[:, 1]

# faz as previsões com dados de treino
Y_pred_train = modelo_rl.predict(X_train)
Y_pred_proba_train = modelo_rl.predict_proba(X_train)[:, 1]

In [ ]:
# plota a matriz de confusão de teste
plot_cm(Y_test, Y_pred, 'Regressão Logística - Teste')

In [ ]:
# plota a matriz de confusão de teste
plot_cm(Y_train, Y_pred_train, 'Regressão Logística - Treino')

In [ ]:
# plota a curva ROC de teste
plot_roc_curve(Y_test, Y_pred_proba, 'Regressão Logística - Teste')

In [ ]:
# plota a curva ROC de teste
plot_roc_curve(Y_train, Y_pred_proba_train, 'Regressão Logística - Treino')

In [ ]:
# plota um histograma com as probabilidades de churn
df_result = pd.DataFrame({'Churn':Y_test, 'Churn_Prob':Y_pred_proba})
sns.histplot(data=df_result,x = 'Churn_Prob',hue='Churn', bins=20, kde=True)

In [ ]:
# plota um histograma com as probabilidades de churn
df_result = pd.DataFrame({'Churn':Y_train, 'Churn_Prob':Y_pred_proba_train})
sns.histplot(data=df_result,x = 'Churn_Prob',hue='Churn', bins=20, kde=True)

#### 4.2 Encontrando o Melhor Threshold

##### 4.2.1 Gradient Boosting

In [ ]:
# faz as previsões com dados de teste
Y_pred = modelo_gb.predict(X_test)
Y_pred_proba = modelo_gb.predict_proba(X_test)[:, 1]

# faz as previsões com dados de treino
Y_pred_train = modelo_gb.predict(X_train)
Y_pred_proba_train = modelo_gb.predict_proba(X_train)[:, 1]

In [ ]:
# inicializa a lista de scores
score = []
max_score = 0
dif_minima = 1

# loop para testar diferentes thresholds
for threshold in np.arange(0.1, 0.9, 0.01):
    # aplica o threshold na probabilidade
    Y_pred_treshold = (Y_pred_proba > threshold).astype(int)

    # calcula as métricas
    vlr_f1_score = f1_score(Y_test, Y_pred_treshold)
    vlr_recall_score = recall_score(Y_test, Y_pred_treshold)
    vlr_precision_score = precision_score(Y_test, Y_pred_treshold)

    # mínimo entre recall e precision
    vlr_min_score = abs(vlr_recall_score - vlr_precision_score)
    if vlr_min_score < dif_minima:
        dif_minima = vlr_min_score
        best_min_threshold = threshold

    # salva o threshold e o score se ele for maior que o máximo
    if vlr_f1_score > max_score:
        max_score = vlr_f1_score
        best_f1_threshold = threshold

    # adiciona o resultado na lista
    score.append((threshold,vlr_f1_score,vlr_recall_score,vlr_precision_score))
print(f'O F1-Score máximo foi de {max_score:.2f} usando o threshold de {best_f1_threshold:.2f}')
print(f'A menor diferença entre precision e recall foi de {dif_minima:.2f} usando o threshold de {best_min_threshold:.2f}')

# threshold final do modelo de boosting
threshold_gb = best_min_threshold

In [ ]:
# plotando o gráfico de F1-Score
df_score = pd.DataFrame(score, columns=['threshold','f1_score','recall_score','precision_score'])
ax = df_score.plot(x='threshold', y=['f1_score','recall_score','precision_score'], figsize=(10,5), grid=True)
ax.axvline(best_f1_threshold, color="red", linestyle="--",label='Melhor F1')
ax.axvline(best_min_threshold, color="black", linestyle="--",label='Precision = Recall')

##### 4.2.2 Regressão Logística

In [ ]:
# faz as previsões com dados de teste
Y_pred = modelo_rl.predict(X_test)
Y_pred_proba = modelo_rl.predict_proba(X_test)[:, 1]

# faz as previsões com dados de treino
Y_pred_train = modelo_rl.predict(X_train)
Y_pred_proba_train = modelo_rl.predict_proba(X_train)[:, 1]

In [ ]:
# inicializa a lista de scores
score = []
max_score = 0
dif_minima = 1

# loop para testar diferentes thresholds
for threshold in np.arange(0.1, 0.9, 0.01):
    # aplica o threshold na probabilidade
    Y_pred_treshold = (Y_pred_proba > threshold).astype(int)

    # calcula as métricas
    vlr_f1_score = f1_score(Y_test, Y_pred_treshold)
    vlr_recall_score = recall_score(Y_test, Y_pred_treshold)
    vlr_precision_score = precision_score(Y_test, Y_pred_treshold)

    # mínimo entre recall e precision
    vlr_min_score = abs(vlr_recall_score - vlr_precision_score)
    if vlr_min_score < dif_minima:
        dif_minima = vlr_min_score
        best_min_threshold = threshold

    # salva o threshold e o score se ele for maior que o máximo
    if vlr_f1_score > max_score:
        max_score = vlr_f1_score
        best_f1_threshold = threshold

    # adiciona o resultado na lista
    score.append((threshold,vlr_f1_score,vlr_recall_score,vlr_precision_score))
print(f'O F1-Score máximo foi de {max_score:.2f} usando o threshold de {best_f1_threshold:.2f}')
print(f'A menor diferença entre precision e recall foi de {dif_minima:.2f} usando o threshold de {best_min_threshold:.2f}')

# threshold final do modelo de regressão
threshold_rl = best_min_threshold

In [ ]:
# plotando o gráfico de F1-Score
df_score = pd.DataFrame(score, columns=['threshold','f1_score','recall_score','precision_score'])
ax = df_score.plot(x='threshold', y=['f1_score','recall_score','precision_score'], figsize=(10,5), grid=True)
ax.axvline(best_f1_threshold, color="red", linestyle="--",label='Melhor F1')
ax.axvline(best_min_threshold, color="black", linestyle="--",label='Precision = Recall')

### 4.3 Fazendo Previsões e métricas finais

##### 4.3.1 Gradient Boosting

In [ ]:
# faz as previsões com dados de teste
Y_pred = modelo_gb.predict(X_test)
Y_pred_proba = modelo_gb.predict_proba(X_test)[:, 1]

# faz as previsões com dados de treino
Y_pred_train = modelo_gb.predict(X_train)
Y_pred_proba_train = modelo_gb.predict_proba(X_train)[:, 1]

# aplica o threshold na probabilidade
Y_pred_treshold = (Y_pred_proba > threshold_gb).astype(int)

In [ ]:
# plota a matriz de confusão de teste
plot_cm(Y_test, Y_pred_treshold, 'Gradient Boosting - Teste com Threshold')

In [ ]:
# plota a curva ROC de teste
plot_roc_curve(Y_test, Y_pred_proba, 'Gradient Boosting - Teste com Threshold')

In [ ]:
# plota um histograma com as probabilidades de churn
df_result = pd.DataFrame({'Churn':Y_test, 'Churn_Prob':Y_pred_proba})
sns.histplot(data=df_result,x = 'Churn_Prob',hue='Churn', bins=20, kde=True)

##### 4.3.2 Regressão Logística

In [ ]:
# faz as previsões com dados de teste
Y_pred = modelo_rl.predict(X_test)
Y_pred_proba = modelo_rl.predict_proba(X_test)[:, 1]

# faz as previsões com dados de treino
Y_pred_train = modelo_rl.predict(X_train)
Y_pred_proba_train = modelo_rl.predict_proba(X_train)[:, 1]

# aplica o threshold na probabilidade
Y_pred_treshold = (Y_pred_proba > threshold_rl).astype(int)

In [ ]:
# plota a matriz de confusão de teste
plot_cm(Y_test, Y_pred_treshold, 'Regressão Logística - Teste com Threshold')

In [ ]:
# plota a curva ROC de teste
plot_roc_curve(Y_test, Y_pred_proba, 'Regressão Logística - Teste com Threshold')

In [ ]:
# plota um histograma com as probabilidades de churn
df_result = pd.DataFrame({'Churn':Y_test, 'Churn_Prob':Y_pred_proba})
sns.histplot(data=df_result,x = 'Churn_Prob',hue='Churn', bins=20, kde=True)